# TF OpenPose Notebook

This is a jupyter-notebook implementation of the tf-pose-estimation/run.py script.

In [1]:
import argparse
import logging
import time

import cv2
import numpy as np

from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [2]:
logger = logging.getLogger('TfPoseEstimator-Video')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)



In [13]:
def main(argString = '--video ./images/video.avi --resolution 656x368 --model inf'):
    
    fps_time = 0
    
    parser = argparse.ArgumentParser(description='tf-pose-estimation Video')
    parser.add_argument('--video', type=str, default='')
    parser.add_argument('--resolution', type=str, default='432x368', help='network input resolution. default=432x368')
    parser.add_argument('--model', type=str, default='mobilenet_thin', help='cmu / mobilenet_thin / mobilenet_v2_large / mobilenet_v2_small')
    parser.add_argument('--show-process', type=bool, default=False,
                        help='for debug purpose, if enabled, speed for inference is dropped.')
    parser.add_argument('--showBG', type=bool, default=True, help='False to show skeleton only.')
    args = parser.parse_args(argString.split())

    logger.debug('initialization %s : %s' % (args.model, get_graph_path(args.model)))
    w, h = model_wh(args.resolution)
    e = TfPoseEstimator(get_graph_path(args.model), target_size=(w, h))
    cap = cv2.VideoCapture(args.video)
    out = cv2.VideoWriter('./images/output.avi', cv2.VideoWriter_fourcc(*'XVID'), 10.0, (int(cap.get(3)), int(cap.get(4))))

    if cap.isOpened() is False:
        print("Error opening video stream or file")
    while cap.isOpened():
        ret_val, image = cap.read()
        
        try:
            humans = e.inference(image, upsample_size=4.0)
        except:
            logger.debug('BREAK on Exception: The image is not valid. Please check your image exists.')
            break
        if not args.showBG:
            image = np.zeros(image.shape)
        image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

        cv2.putText(image, "FPS: %f" % (1.0 / (time.time() - fps_time)), (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        logger.debug("FPS: %f" % (1.0 / (time.time() - fps_time)))
        #cv2.imshow('tf-pose-estimation result', image)
        out.write(image)
        fps_time = time.time()
        #if cv2.waitKey(1) == 27:
        #    break
    cap.release()
    out.release()
    cv2.destroyAllWindows()

   


In [14]:
main('--video ./images/video.avi --resolution 656x368 --model inf')

[2020-07-14 08:22:59,206] [TfPoseEstimator-Video] [DEBUG] initialization inf : /home/ubuntu/openpose/tf-pose-estimation/models/graph/inf/graph_opt.pb
[2020-07-14 08:22:59,206] [TfPoseEstimator-Video] [DEBUG] initialization inf : /home/ubuntu/openpose/tf-pose-estimation/models/graph/inf/graph_opt.pb
2020-07-14 08:22:59,206 DEBUG initialization inf : /home/ubuntu/openpose/tf-pose-estimation/models/graph/inf/graph_opt.pb
[2020-07-14 08:22:59,208] [TfPoseEstimator] [INFO] loading graph from /home/ubuntu/openpose/tf-pose-estimation/models/graph/inf/graph_opt.pb(default size=656x368)
2020-07-14 08:22:59,208 INFO loading graph from /home/ubuntu/openpose/tf-pose-estimation/models/graph/inf/graph_opt.pb(default size=656x368)


TfPoseEstimator/preprocess_subtract/y
TfPoseEstimator/preprocess_divide/y
TfPoseEstimator/image
TfPoseEstimator/preprocess_subtract/perm
TfPoseEstimator/preprocess_divide
TfPoseEstimator/preprocess_subtract_PreTransposeNHWC2NCHW
TfPoseEstimator/preprocess_subtract_PreCastFloat32ToFlot16
TfPoseEstimator/preprocess_subtract
TfPoseEstimator/conv5_2_CPM_L1/weights/neuron_op_ed41d2deb8c54255
TfPoseEstimator/Openpose/concat_stage7
strided_slice/stack
strided_slice/stack_1
strided_slice/stack_2
strided_slice
strided_slice_1/stack
strided_slice_1/stack_1
strided_slice_1/stack_2
strided_slice_1
upsample_size
strided_slice_2/stack
strided_slice_2/stack_1
strided_slice_2/stack_2
strided_slice_2
upsample_heatmat
strided_slice_3/stack
strided_slice_3/stack_1
strided_slice_3/stack_2
strided_slice_3
upsample_pafmat
smoothing/Const
smoothing/gauss_weight
smoothing/gauss_weight/Assign
smoothing/gauss_weight/read
smoothing/depthwise/Shape
smoothing/depthwise/dilation_rate
smoothing/depthwise
max_pool/di

[2020-07-14 08:23:05,028] [TfPoseEstimator-Video] [DEBUG] FPS: 0.000000
[2020-07-14 08:23:05,028] [TfPoseEstimator-Video] [DEBUG] FPS: 0.000000
2020-07-14 08:23:05,028 DEBUG FPS: 0.000000
[2020-07-14 08:23:05,297] [TfPoseEstimator-Video] [DEBUG] FPS: 3.822449
[2020-07-14 08:23:05,297] [TfPoseEstimator-Video] [DEBUG] FPS: 3.822449
2020-07-14 08:23:05,297 DEBUG FPS: 3.822449
[2020-07-14 08:23:05,557] [TfPoseEstimator-Video] [DEBUG] FPS: 3.969578
[2020-07-14 08:23:05,557] [TfPoseEstimator-Video] [DEBUG] FPS: 3.969578
2020-07-14 08:23:05,557 DEBUG FPS: 3.969578
[2020-07-14 08:23:05,819] [TfPoseEstimator-Video] [DEBUG] FPS: 3.945709
[2020-07-14 08:23:05,819] [TfPoseEstimator-Video] [DEBUG] FPS: 3.945709
2020-07-14 08:23:05,819 DEBUG FPS: 3.945709
[2020-07-14 08:23:06,079] [TfPoseEstimator-Video] [DEBUG] FPS: 3.958160
[2020-07-14 08:23:06,079] [TfPoseEstimator-Video] [DEBUG] FPS: 3.958160
2020-07-14 08:23:06,079 DEBUG FPS: 3.958160
[2020-07-14 08:23:06,340] [TfPoseEstimator-Video] [DEBUG] FP

In [17]:
from IPython.display import Video

Video('./images/output.avi',filename='./images/output.avi')